In [1]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import pyscipopt as pyscip
import pickle as pkl

from generate_data import *
from dump_data import *

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


ModuleNotFoundError: No module named 'ecole'

# Generate Problem and collect features

In [ ]:
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }
generate_dataset(scip_parameters,path = "DataSet/",\
                 nb_cons = [100,200,300,400,500],nb_var = [1,1.5,2],density = [0.1,0.15,0.2])

# Collect labels

In [ ]:

for row in [100,200,300,400,500]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
#                 #save label
                with HiddenPrints():
                    problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
                    solver = ecole.scip.Model.from_file("DataSet/"+problme_name+"/problem.lp")
                    aspyscip = solver.as_pyscipopt()
                    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                    aspyscip.optimize()
                    gapList.append(aspyscip.getGap())
                    dumpSolution_Ecole("DataSet/"+problme_name+"/label.json",aspyscip)
                    
gap = np.array(gapList)
print(np.mean(gap))
print(np.where(gap != 0))

In [ ]:
import ast
instance = ecole.scip.Model.from_file("DataSet/set_cover_{100*100_0.1_35}/problem.lp")
aspyscip = instance.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
bestsol = aspyscip.getBestSol()
print(bestsol)
solutionPool = []
for sol in aspyscip.getSols():
    solutionPool.append(np.array(list(ast.literal_eval(sol.__repr__()).values())).astype(int))
solutions = {
    "Best_Solution":bestsol.__str__(),
    "Solution_Pool":aspyscip.getSols().__str__()
}
print(np.around(list(ast.literal_eval(aspyscip.getSols()[0].__repr__()).values()),0).astype(int).tolist())

In [ ]:
#scip
with HiddenPrints():
    model_scip = pyscip.scip.Model()
    model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    model_scip.readProblem(filename="debugFile/set_cover_test.lp",extension = "lp")
    # model_scip.readProblem(filename="debugFile/debug_diffc.lp",extension = "lp")

model_scip.optimize()
# time.sleep(1)
print("\n",model_scip.getBestSol())

# print("Ecole*********************")
# # with HiddenPrints():
# solver = ecole.scip.Model.from_file("debugFile/debug.lp")
# aspyscip = solver.as_pyscipopt()
# aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
# aspyscip.optimize()
# gap = aspyscip.getGap()
# sol = aspyscip.getBestSol()
# print(sol,gap)

# #gurobi
# print("Gurobi*********************")
# with HiddenPrints():
#     model = gb.read("debugFile/debug.lp")
#     model.optimize()
# for var in model.getVars():
#     print(f"{var.varName}: {round(var.X, 3)}")



In [ ]:
original_indice = obs.variable_features[:,-1]
print(original_indice)

In [ ]:
dumpEdgeFeatures("debugFile/test_EDGEFE.json",obs.edge_features,original_indice)
dumpRowFeatures("debugFile/test_ROWFE.json",obs.row_features)
dumpVariableFeatures("debugFile/test_VarFE.json",obs.variable_features)

In [ ]:

aspyscip = instance.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
dumpSolution_Ecole("debugFile/label.json",aspyscip)

In [ ]:
#bias
print(obs.RowFeatures.value)
print(action_set)

In [ ]:
aspyscip = instance.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
gapList.append(aspyscip.getGap())
dumpSolution_Ecole("debugFile/label_test.json",aspyscip)

In [ ]:
print(env.model)
print(instance)
aspyscip = env.model.as_pyscipopt()
variable = aspyscip.getVars()
for var in variable:
    print(var.getObj() )

In [ ]:
for row in [100,150,200]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in range(1,2):
#                 #save label
                problme_name = \
                        "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
#                 solver = ecole.scip.Model.from_file("DataSet/problem/"+problme_name+".lp")
#                 aspyscip = solver.as_pyscipopt()
#                 aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
#                 aspyscip.optimize()
                model_scip = pyscip.scip.Model()
                model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                model_scip.readProblem(filename="DataSet/problem/"+problme_name+".lp")
                model_scip.optimize()